In [12]:
import csv
import pandas as pd
pd.options.display.max_colwidth = 200
import PySimpleGUI as sg
from tqdm import tqdm
import tensorflow_hub as hub
import numpy as np

def gui(data):
    # GUI for inputing a sentence to search and outputing results
    sg.theme('DARKBLUE4')
    layout = [[sg.Input(key='-IN-')],
              [sg.Button('Search', bind_return_key=True)],
             [ sg.Text(size=(100,30), key='-OUTPUT-')]]

    window = sg.Window('NLP1', layout)
    
    while True:
        event, values = window.read()

        if event in  (None, 'Exit'):
            break

        if event == 'Search':
            window.TKroot.title('LOADING')
            result = search(data, values['-IN-'])
            if len(result) == 0:
                print("lmao")
                window['-OUTPUT-'].update("BAD INPUT: check letter count >=5 and wordcount >=3")
                window.TKroot.title('NLP1')
            else:
                outputs = "BEST:\n" + format_output(result[0]) + "\nMIDDLE:\n" + format_output(result[1]) + "\nWORST:\n" + format_output(result[2])
                window['-OUTPUT-'].update(outputs)
                window.TKroot.title('NLP1')

    window.close()
    
def format_output(result):
    result = result.values
    return '\n'.join(
                    ["In " + str(stype) + " " + str(value) + ": " + title + "\n" + link
                     for title, article, value, link, stype in 
                     zip(
                         [i[0] for i in result],
                         [i[1] for i in result],
                         [i[2] for i in result],
                         [i[3] for i in result],
                         [i[4] for i in result]
                     )])
    
def search(indata, tosearch):
    # Method to complete a semantic search
    # Output is 10 most similar articles
    # Final score is a maximum of similarities between article and title
    
    if len(tosearch) < 5:
        print("BAD INPUT: letter count < 5")
        return pd.DataFrame([])
    if len(tosearch.split()) < 3:
        print("BAD INPUT: word count < 3")
        return pd.DataFrame([])

    search = embed([tosearch])
    df = indata.copy()
    
    search_title = []
    for i in df.embed_title.values:
        search_title.append(round(np.inner(i,search)[0][0]*100,2))
    df["search_title"] = search_title
    
    search_article = []
    for i in df.embed_article.values:
        search_article.append(round(np.inner(i,search)[0][0]*100,2))
    df["search_article"]s= search_article
    
    scores = []
    types = []
    for t,a in zip(search_title, search_article):
        scores.append(max(t,a))
        if t >= a:
            types.append("title")
        else:
            types.append("article")
        
    df["score"] = scores
    df["type"] = types
    
    df = df.sort_values(by=['score'], ascending=False)
    df = df[["title","article","score","url", "type"]]
    
    first = df.head(3)
    middle = df.iloc[int((len(df))/2)-2:int((len(df))/2)+1]
    last = df.tail(3)
    return (first, middle, last)

### Reading  data

In [2]:
data = pd.read_csv("SCRAPED.csv", encoding="UTF-8", sep = "¬", na_filter = True, engine='python')

data["article"] = [str(i).strip() for i in data.article]
data["title"] = [str(i).strip() for i in data.title]

In [15]:
data

,title,article,date,url,embed_title,embed_article
0,IN BRIEF: Reabold Resources' Corallian to assess options going forward,Reabold Resources PLC - invests in upstream oil & gas projects - Says its 49.99%-owned Corallian Energy Ltd will formally review various strategic options available to maximise value for all its s...,"Wed, 20th Oct 2021 12:06",https://www.lse.co.uk/news/in-brief-reabold-resources-corallian-to-assess-options-going-forward-jyya29jf61w5ywr.html,"((tf.Tensor(0.0056207324, shape=(), dtype=float32), tf.Tensor(-0.045668818, shape=(), dtype=float32), tf.Tensor(-0.07929104, shape=(), dtype=float32), tf.Tensor(-0.033190787, shape=(), dtype=float...","((tf.Tensor(-0.03658915, shape=(), dtype=float32), tf.Tensor(-0.07209045, shape=(), dtype=float32), tf.Tensor(-0.06893418, shape=(), dtype=float32), tf.Tensor(-0.022192858, shape=(), dtype=float32..."
1,"Rugby-No overall link between concussion, cognitive function in ex-players over 50 - study",Oct 20 (Reuters) - Retired male rugby players aged 50 and over who have had at least three concussions in their careers have no worse average cognitive function overall compared with those who sus...,"Wed, 20th Oct 2021 12:00",https://www.lse.co.uk/news/rugby-no-overall-link-between-concussion-cognitive-function-in-ex-players-over-50-study-uybql2he43vb4u2.html,"((tf.Tensor(0.054410227, shape=(), dtype=float32), tf.Tensor(0.055596028, shape=(), dtype=float32), tf.Tensor(-0.009993697, shape=(), dtype=float32), tf.Tensor(0.0022165384, shape=(), dtype=float3...","((tf.Tensor(0.017382314, shape=(), dtype=float32), tf.Tensor(0.0545723, shape=(), dtype=float32), tf.Tensor(-0.0579802, shape=(), dtype=float32), tf.Tensor(-0.058005314, shape=(), dtype=float32), ..."
2,Europe midday: Shares flat on mixed earnings; Husqvarna powers ahead,(Sharecast News) - European shares were barely moving the needle on Wednesday as investors digested a mixed bag of corporate earnings news. The pan-European Stoxx 600 index was up 0.05% in early d...,"Wed, 20th Oct 2021 12:00",https://www.lse.co.uk/news/europe-midday-shares-flat-on-mixed-earnings-husqvarna-powers-ahead-3escbhex9yeqaee.html,"((tf.Tensor(-0.006834281, shape=(), dtype=float32), tf.Tensor(-0.04403242, shape=(), dtype=float32), tf.Tensor(-0.065756775, shape=(), dtype=float32), tf.Tensor(0.072913125, shape=(), dtype=float3...","((tf.Tensor(-0.056410607, shape=(), dtype=float32), tf.Tensor(-0.058924444, shape=(), dtype=float32), tf.Tensor(-0.019915372, shape=(), dtype=float32), tf.Tensor(-0.00557921, shape=(), dtype=float..."
3,Britain's Lloyds Bank to close another 48 branches,"LONDON, Oct 20 (Reuters) - Lloyds Banking Group will close a further 48 branches across England and Wales, the British lender said on Wednesday, as it seeks to further cut costs by trimming its ph...","Wed, 20th Oct 2021 12:36",https://www.lse.co.uk/news/britains-lloyds-bank-to-close-another-48-branches-4ecdyb4idy46pu7.html,"((tf.Tensor(-0.027998546, shape=(), dtype=float32), tf.Tensor(-0.007926836, shape=(), dtype=float32), tf.Tensor(-0.053467724, shape=(), dtype=float32), tf.Tensor(0.037366595, shape=(), dtype=float...","((tf.Tensor(-0.018615384, shape=(), dtype=float32), tf.Tensor(-0.050030723, shape=(), dtype=float32), tf.Tensor(-0.033339996, shape=(), dtype=float32), tf.Tensor(-0.03784739, shape=(), dtype=float..."
4,Eurozone inflation rises as expected in September,"(Sharecast News) - Eurozone inflation rose as expected in September, according to data released on Wednesday by Eurostat. Consumer price inflation increased to 3.4% from 3.0% in August, in line wi...","Wed, 20th Oct 2021 12:40",https://www.lse.co.uk/news/eurozone-inflation-rises-as-expected-in-september-8q4hxuaqiwgjiup.html,"((tf.Tensor(0.01069256, shape=(), dtype=float32), tf.Tensor(0.05060229, shape=(), dtype=float32), tf.Tensor(0.015388406, shape=(), dtype=float32), tf.Tensor(0.030949226, shape=(), dtype=float32), ...","((tf.Tensor(-0.026597507, shape=(), dtype=float32), tf.Tensor(-0.016724959, shape=()

### Vectorizing title and article

In [3]:
 #If a model from local computer is needed due to network problems
embed = hub.KerasLayer(
    'C:\\Users\\user\\Desktop\\VU\\5 pusmetis\\Natūralios kalbos apdorojimas\\Uzduotys\\model'
)
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [4]:
embed_title = []
for i in tqdm(data.title.values):
    embed_title.append(embed([i]))

data["embed_title"] = embed_title

100%|███████████████████████████████████████████████████████████████████████████████| 225/225 [00:00<00:00, 259.04it/s]


In [5]:
embed_article = []
for i in tqdm(data.article.values):
    embed_article.append(embed([i]))
    
data["embed_article"] = embed_article

100%|███████████████████████████████████████████████████████████████████████████████| 225/225 [00:00<00:00, 265.54it/s]


In [6]:
data.head(3)

,title,article,date,url,embed_title,embed_article
0,IN BRIEF: Reabold Resources' Corallian to assess options going forward,Reabold Resources PLC - invests in upstream oil & gas projects - Says its 49.99%-owned Corallian Energy Ltd will formally review various strategic options available to maximise value for all its s...,"Wed, 20th Oct 2021 12:06",https://www.lse.co.uk/news/in-brief-reabold-resources-corallian-to-assess-options-going-forward-jyya29jf61w5ywr.html,"((tf.Tensor(0.0056207324, shape=(), dtype=float32), tf.Tensor(-0.045668818, shape=(), dtype=float32), tf.Tensor(-0.07929104, shape=(), dtype=float32), tf.Tensor(-0.033190787, shape=(), dtype=float...","((tf.Tensor(-0.03658915, shape=(), dtype=float32), tf.Tensor(-0.07209045, shape=(), dtype=float32), tf.Tensor(-0.06893418, shape=(), dtype=float32), tf.Tensor(-0.022192858, shape=(), dtype=float32..."
1,"Rugby-No overall link between concussion, cognitive function in ex-players over 50 - study",Oct 20 (Reuters) - Retired male rugby players aged 50 and over who have had at least three concussions in their careers have no worse average cognitive function overall compared with those who sus...,"Wed, 20th Oct 2021 12:00",https://www.lse.co.uk/news/rugby-no-overall-link-between-concussion-cognitive-function-in-ex-players-over-50-study-uybql2he43vb4u2.html,"((tf.Tensor(0.054410227, shape=(), dtype=float32), tf.Tensor(0.055596028, shape=(), dtype=float32), tf.Tensor(-0.009993697, shape=(), dtype=float32), tf.Tensor(0.0022165384, shape=(), dtype=float3...","((tf.Tensor(0.017382314, shape=(), dtype=float32), tf.Tensor(0.0545723, shape=(), dtype=float32), tf.Tensor(-0.0579802, shape=(), dtype=float32), tf.Tensor(-0.058005314, shape=(), dtype=float32), ..."
2,Europe midday: Shares flat on mixed earnings; Husqvarna powers ahead,(Sharecast News) - European shares were barely moving the needle on Wednesday as investors digested a mixed bag of corporate earnings news. The pan-European Stoxx 600 index was up 0.05% in early d...,"Wed, 20th Oct 2021 12:00",https://www.lse.co.uk/news/europe-midday-shares-flat-on-mixed-earnings-husqvarna-powers-ahead-3escbhex9yeqaee.html,"((tf.Tensor(-0.006834281, shape=(), dtype=float32), tf.Tensor(-0.04403242, shape=(), dtype=float32), tf.Tensor(-0.065756775, shape=(), dtype=float32), tf.Tensor(0.072913125, shape=(), dtype=float3...","((tf.Tensor(-0.056410607, shape=(), dtype=float32), tf.Tensor(-0.058924444, shape=(), dtype=float32), tf.Tensor(-0.019915372, shape=(), dtype=float32), tf.Tensor(-0.00557921, shape=(), dtype=float..."


## Additional exercise

### GUI

In [16]:
gui(data)